In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-04-24"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
9725,2024-04-24,Eua Nba,22:30,Oklahoma City Thunder,New Orleans Pelicans,1.32,3.61,211.5,1.93,1.93,-7.5,1.93,7.50,lKTjTKKP,0.757576,0.277008,0.518135,0.518135,0.034584,0.4,0.4,NaN,NaN,225.506,40.576908,0.179937,231.744,29.126502,0.125684,194.404,178.556,182.56,274.05,0.0,0.0,0.0,0.0,0.656907,0.000000,0.835331,2.83,0.566,0.565371,0.717999,0.9,0.182001,1.23,0.246,10.609756,0.576343,0.7,0.123657
9726,2024-04-24,Europa Euroliga,14:00,Monaco,Fenerbahce,1.67,2.28,159.5,1.91,1.91,-3.5,1.95,2.90,Stu5P0Ts,0.598802,0.438596,0.523560,0.523560,0.037399,0.2,0.2,NaN,NaN,118.162,40.093339,0.339308,162.224,35.092829,0.216323,106.280,167.082,94.01,221.94,0.0,0.0,0.0,0.0,0.218398,0.000000,0.277011,1.26,0.252,2.658730,0.775217,1.0,0.224783,-3.59,-0.718,-1.782730,0.491134,0.2,-0.291134
9727,2024-04-24,Austrália Nbl1 Sul,05:00,Frankston F,Dandenong Rangers F,1.64,2.18,152.5,1.87,1.79,-3.5,1.98,1.93,lWgGxmms,0.609756,0.458716,0.534759,0.558659,0.068472,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,184.80,0.0,0.0,0.0,0.0,0.199915,0.030912,0.018085,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
9728,2024-04-24,Europa Copa Europeia Da Fiba,13:00,Bahcesehir Kol.,Chemnitz,2.00,1.73,162.5,1.80,1.86,-1.5,2.01,1.89,tnIJrWLk,0.500000,0.578035,0.555556,0.537634,0.078035,0.8,0.6,NaN,NaN,116.312,8.437279,0.072540,158.228,98.604718,0.623181,115.654,174.948,109.12,332.00,0.0,0.0,0.0,0.0,0.102369,0.023184,0.043514,0.72,0.144,6.944444,0.000000,0.0,0.000000,-0.44,-0.088,-8.295455,0.600318,0.7,0.099682
9729,2024-04-24,Filipinas Copa Das Filipinas,05:30,NorthPort,Terrafirma Dyip,1.61,2.25,204.5,1.87,1.87,-4.5,2.05,3.00,KQ8NDWfp,0.621118,0.444444,0.534759,0.534759,0.065562,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,266.60,0.00,0.0,0.0,0.0,0.0,0.234481,0.000000,0.266040,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
9730,2024-04-24,França Lnb,15:00,Boulogne-Levallois,JL Bourg,6.55,1.12,167.5,1.83,1.93,13.5,1.88,1.01,zm843z7O,0.152672,0.892857,0.546448,0.518135,0.045529,0.4,0.4,NaN,NaN,429.968,284.508075,0.661696,156.474,26.841430,0.171539,302.954,154.420,178.50,183.26,0.0,0.0,0.0,0.0,1.001197,0.037612,0.425732,-1.81,-0.362,-15.331492,0.000000,0.0,0.000000,-0.25,-0.050,-2.400000,0.000000,0.0,0.000000
9731,2024-04-24,Geórgia Superliga,07:00,Kavkasia,TSU,2.09,1.69,175.5,1.85,1.85,1.5,1.96,1.53,IiktEnAM,0.478469,0.591716,0.540541,0.540541,0.070185,0.0,0.6,NaN,NaN,NaN,NaN,NaN,129.004,28.636520,0.221982,0.000,148.000,124.74,154.70,0.0,0.0,0.0,0.0,0.149652,0.000000,0.174244,0.00,0.000,inf,0.000000,0.0,0.000000,-0.60,-0.120,-5.750000,0.000000,0.0,0.000000
9732,2024-04-24,Geórgia Superliga,13:00,Iverioni,Gurjaani Delta,1.74,1.99,159.5,1.85,1.85,-2.5,2.01,2.39,IR5kuP8L,0.574713,0.502513,0.540541,0.540541,0.077225,0.0,0.0,NaN,NaN,NaN,NaN,NaN,127.286,13.325280,0.104688,0.000,0.000,0.00,142.76,0.0,0.0,0.0,0.0,0.094786,0.000000,0.122137,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
9733,2024-04-24,Hungria Nacional,13:00,Kormend,Atomeromu Paks,1.70,2.05,171.5,1.85,1.85,-2.5,1.95,2.48,0rHvxe1k,0.588235,0.487805,0.540541,0.540541,0.076040,0.8,0.6,NaN,NaN,262.124,86.137496,0.328614,218.610,149.825934,0.685357,161.198,186.990,329.60,196.35,0.0,0.0,0.0,0.0,0.131993,0.000000,0.169195,-0.25,-0.050,-14.000000,0.000000,0.0,0.000000,-0.15,-0.030,-35.000000,0.000000,0.0,0.000000
9734,2024-04-24,Nova Zelândia Nbl,23:00,Otag

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,22:30,Eua Nba,Oklahoma City Thunder,New Orleans Pelicans,211.5,1.93,1.0,Over
1,14:00,Europa Euroliga,Monaco,Fenerbahce,159.5,1.91,1.0,Over
2,05:00,Austrália Nbl1 Sul,Frankston F,Dandenong Rangers F,152.5,1.87,1.0,Over
3,05:30,Filipinas Copa Das Filipinas,NorthPort,Terrafirma Dyip,204.5,1.87,1.0,Over
4,13:00,Geórgia Superliga,Iverioni,Gurjaani Delta,159.5,1.85,1.0,Over
5,13:00,Hungria Nacional,Kormend,Atomeromu Paks,171.5,1.85,1.0,Over
6,23:00,Nova Zelândia Nbl,Otago Nuggets,Canterbury Rams,185.5,1.87,1.0,Over
7,13:00,República Tcheca Nbl,Usti n. Labem,Pardubice,160.5,1.83,1.0,Over
